# 往期回顾

In [91]:
from IPython.display import Markdown as md
from  matplotlib import pyplot as plt
import sqlite3
import time
import pandas as pd
import json

df=pd.read_csv("train.csv")
# 将表格中的所有nan换成null值
df=df.where(pd.notnull(df),None)
# 制作两个新列表
passengerInfo_cols=['Name','Sex','Age','SibSp','Parch']
ticketInfo_cols=['Pclass','Ticket','Fare','Cabin','Embarked']

#两列分别以一个json字符串的形式存储对应的数据
df["passengerInfo"]=df[passengerInfo_cols].apply(lambda x: json.dumps(x.to_dict()),axis=1)
df['ticketInfo']=df[ticketInfo_cols].apply(lambda x:json.dumps(x.to_dict()),axis=1)


# 过滤原来的列
df=df.drop(passengerInfo_cols,axis=1)
df=df.drop(ticketInfo_cols,axis=1)


# 转换数据为str
df['passengerInfo']=df['passengerInfo'].astype(str)
df['ticketInfo']=df['ticketInfo'].astype(str)


# 保存到example_table_json的表中
with sqlite3.connect('example_db') as conn:
    df.to_sql('example_table_json',con=conn,if_exists='replace',index=False)

connection=sqlite3.connect('example_db')


In [92]:
query=""" 
SELECT 
    PassengerId
    ,passengerInfo 
    ,ticketInfo
    ,json_insert(ticketInfo,'$.SibSp',json_extract(passengerInfo,'$.SibSp')
                            ,'$.Parch',json_extract(passengerInfo,'$.Parch')) AS new_passengerInfo
    ,json_remove(passengerInfo,'$.SibSp','$.Parch')  AS new_ticketInfo
FROM example_table_json
"""
result=connection.execute(query).fetchall()
result

[(1,
  '{"Name": "Braund, Mr. Owen Harris", "Sex": "male", "Age": 22.0, "SibSp": 1, "Parch": 0}',
  '{"Pclass": 3, "Ticket": "A/5 21171", "Fare": 7.25, "Cabin": null, "Embarked": "S"}',
  '{"Pclass":3,"Ticket":"A/5 21171","Fare":7.25,"Cabin":null,"Embarked":"S","SibSp":1,"Parch":0}',
  '{"Name":"Braund, Mr. Owen Harris","Sex":"male","Age":22.0}'),
 (2,
  '{"Name": "Cumings, Mrs. John Bradley (Florence Briggs Thayer)", "Sex": "female", "Age": 38.0, "SibSp": 1, "Parch": 0}',
  '{"Pclass": 1, "Ticket": "PC 17599", "Fare": 71.2833, "Cabin": "C85", "Embarked": "C"}',
  '{"Pclass":1,"Ticket":"PC 17599","Fare":71.2833,"Cabin":"C85","Embarked":"C","SibSp":1,"Parch":0}',
  '{"Name":"Cumings, Mrs. John Bradley (Florence Briggs Thayer)","Sex":"female","Age":38.0}'),
 (3,
  '{"Name": "Heikkinen, Miss. Laina", "Sex": "female", "Age": 26.0, "SibSp": 0, "Parch": 0}',
  '{"Pclass": 3, "Ticket": "STON/O2. 3101282", "Fare": 7.925, "Cabin": null, "Embarked": "S"}',
  '{"Pclass":3,"Ticket":"STON/O2. 31012

In [93]:
query="""  
SELECT  
    PassengerId,
    passengerInfo,
    ticketInfo,
    json_insert(ticketInfo,'$.SibSp',json_extract(passengerInfo,'$.SibSp'),
    '$.Parch',json_extract(passengerInfo,'$.Parch')) as new_passengerInfo,
    json_remove(passengerInfo,'$.SibSp','$.Parch') as new_ticketInfo
from example_table_json
    

"""
result=connection.execute(query).fetchall()
result

[(1,
  '{"Name": "Braund, Mr. Owen Harris", "Sex": "male", "Age": 22.0, "SibSp": 1, "Parch": 0}',
  '{"Pclass": 3, "Ticket": "A/5 21171", "Fare": 7.25, "Cabin": null, "Embarked": "S"}',
  '{"Pclass":3,"Ticket":"A/5 21171","Fare":7.25,"Cabin":null,"Embarked":"S","SibSp":1,"Parch":0}',
  '{"Name":"Braund, Mr. Owen Harris","Sex":"male","Age":22.0}'),
 (2,
  '{"Name": "Cumings, Mrs. John Bradley (Florence Briggs Thayer)", "Sex": "female", "Age": 38.0, "SibSp": 1, "Parch": 0}',
  '{"Pclass": 1, "Ticket": "PC 17599", "Fare": 71.2833, "Cabin": "C85", "Embarked": "C"}',
  '{"Pclass":1,"Ticket":"PC 17599","Fare":71.2833,"Cabin":"C85","Embarked":"C","SibSp":1,"Parch":0}',
  '{"Name":"Cumings, Mrs. John Bradley (Florence Briggs Thayer)","Sex":"female","Age":38.0}'),
 (3,
  '{"Name": "Heikkinen, Miss. Laina", "Sex": "female", "Age": 26.0, "SibSp": 0, "Parch": 0}',
  '{"Pclass": 3, "Ticket": "STON/O2. 3101282", "Fare": 7.925, "Cabin": null, "Embarked": "S"}',
  '{"Pclass":3,"Ticket":"STON/O2. 31012

In [94]:
query=""" 
SELECT
    PassengerId,
    passengerInfo,
    ticketInfo,
    json_set(ticketInfo,'$.Name',json_extract(passengerInfo,'$.Name'),
    '$.PassengerId',json_extract(passengerInfo,'$.PassengerId'))
from example_table_json

"""
result = connection.execute(query).fetchone()
result

(1,
 '{"Name": "Braund, Mr. Owen Harris", "Sex": "male", "Age": 22.0, "SibSp": 1, "Parch": 0}',
 '{"Pclass": 3, "Ticket": "A/5 21171", "Fare": 7.25, "Cabin": null, "Embarked": "S"}',
 '{"Pclass":3,"Ticket":"A/5 21171","Fare":7.25,"Cabin":null,"Embarked":"S","Name":"Braund, Mr. Owen Harris","PassengerId":null}')

In [95]:
query="""  
SELECT 
    json_extract(passengerInfo,'$.Age') as Age,
    json_extract(json_group_array(PassengerId),'$[#-2]'),
    json_extract(json_group_array(PassengerId),'$[2]') 

from example_table_json
where survived==1  and json_extract(passengerInfo,'$.Age') is not null
and age >40
group by json_extract(passengerInfo,'$.Age')
"""
result = connection.execute(query).fetchone()
result

(41.0, 273, None)

In [96]:
query = '''
SELECT 
    json_extract(passengerInfo,'$.Age') AS Age
    ,json_extract(json_group_array(PassengerId),'$[#-2]')     
    ,json_extract(json_group_array(PassengerId),'$[2]')     
FROM example_table_json
WHERE Survived==1 AND json_extract(passengerInfo,'$.Age') IS NOT NULL
      AND Age>40

GROUP BY json_extract(passengerInfo,'$.Age') 
'''
result = connection.execute(query).fetchall()
result

[(41.0, 273, None),
 (42.0, 708, 433),
 (43.0, None, None),
 (44.0, 415, 524),
 (45.0, 707, 441),
 (47.0, None, None),
 (48.0, 755, 646),
 (49.0, 600, 600),
 (50.0, 527, 459),
 (51.0, 766, None),
 (52.0, 592, 821),
 (53.0, None, None),
 (54.0, 514, 775),
 (55.0, None, None),
 (56.0, 648, None),
 (58.0, 196, 269),
 (60.0, 367, None),
 (62.0, 571, None),
 (63.0, 276, None),
 (80.0, None, None)]

In [97]:
query=""" 
SELECT
    json_extract(passengerInfo,'$.Age') as Age,
    sum(json_extract(ticketInfo,'$.Fare')) /json_array_length(json_group_array(PassengerId)) as average_fare
    from example_table_json
where survived==1 and age is not null  and age>40
group by age
    
"""
result = connection.execute(query).fetchall()
result

[(41.0, 77.0),
 (42.0, 59.72778333333333),
 (43.0, 211.3375),
 (44.0, 31.208333333333332),
 (45.0, 47.84334),
 (47.0, 52.5542),
 (48.0, 47.63473333333334),
 (49.0, 62.17295000000001),
 (50.0, 85.63416),
 (51.0, 52.254149999999996),
 (52.0, 67.42223333333334),
 (53.0, 51.4792),
 (54.0, 53.555566666666664),
 (55.0, 16.0),
 (56.0, 59.32915),
 (58.0, 108.84443333333336),
 (60.0, 77.225),
 (62.0, 45.25),
 (63.0, 43.7729),
 (80.0, 30.0)]

In [98]:
query = '''
SELECT 
    json_extract(passengerInfo,'$.Age') AS Age
    ,SUM(json_extract(ticketInfo,'$.Fare')) / json_array_length(json_group_array(PassengerId)) AS averager_fare
FROM example_table_json
WHERE Survived==1 AND json_extract(passengerInfo,'$.Age') IS NOT NULL
      AND Age>40
GROUP BY json_extract(passengerInfo,'$.Age') 
'''
result = connection.execute(query).fetchall()
result

[(41.0, 77.0),
 (42.0, 59.72778333333333),
 (43.0, 211.3375),
 (44.0, 31.208333333333332),
 (45.0, 47.84334),
 (47.0, 52.5542),
 (48.0, 47.63473333333334),
 (49.0, 62.17295000000001),
 (50.0, 85.63416),
 (51.0, 52.254149999999996),
 (52.0, 67.42223333333334),
 (53.0, 51.4792),
 (54.0, 53.555566666666664),
 (55.0, 16.0),
 (56.0, 59.32915),
 (58.0, 108.84443333333336),
 (60.0, 77.225),
 (62.0, 45.25),
 (63.0, 43.7729),
 (80.0, 30.0)]

In [99]:
query=""" 
SELECT
    json_extract(PasengerInfo,'$.Age') as Age,
    json_group_array(PassengerId)
from example_table_json
where json_extract(passengerInfo,'$.Age') IS NOT NULL
group by json_extract(passengerInfo,'$.Age')
having json_array_length(json_group_array(PassengerId)) >=3   
"""



In [100]:
query=''' 
SELECT * 
from example_table_json
'''
result=connection.execute(query).fetchall()
result

[(1,
  0,
  '{"Name": "Braund, Mr. Owen Harris", "Sex": "male", "Age": 22.0, "SibSp": 1, "Parch": 0}',
  '{"Pclass": 3, "Ticket": "A/5 21171", "Fare": 7.25, "Cabin": null, "Embarked": "S"}'),
 (2,
  1,
  '{"Name": "Cumings, Mrs. John Bradley (Florence Briggs Thayer)", "Sex": "female", "Age": 38.0, "SibSp": 1, "Parch": 0}',
  '{"Pclass": 1, "Ticket": "PC 17599", "Fare": 71.2833, "Cabin": "C85", "Embarked": "C"}'),
 (3,
  1,
  '{"Name": "Heikkinen, Miss. Laina", "Sex": "female", "Age": 26.0, "SibSp": 0, "Parch": 0}',
  '{"Pclass": 3, "Ticket": "STON/O2. 3101282", "Fare": 7.925, "Cabin": null, "Embarked": "S"}'),
 (4,
  1,
  '{"Name": "Futrelle, Mrs. Jacques Heath (Lily May Peel)", "Sex": "female", "Age": 35.0, "SibSp": 1, "Parch": 0}',
  '{"Pclass": 1, "Ticket": "113803", "Fare": 53.1, "Cabin": "C123", "Embarked": "S"}'),
 (5,
  0,
  '{"Name": "Allen, Mr. William Henry", "Sex": "male", "Age": 35.0, "SibSp": 0, "Parch": 0}',
  '{"Pclass": 3, "Ticket": "373450", "Fare": 8.05, "Cabin": null

# join技巧

In [101]:
from IPython.display import Markdown as md 
import sqlite3
import pandas as pd
import numpy as np
import json
from copy import deepcopy


In [102]:
df=pd.read_csv('train.csv')
df=df.where(pd.notnull(df),None)
with sqlite3.connect('example_db') as conn:
    df.to_sql('example_table',con=conn,if_exists='replace',index=False)

new_df=deepcopy(df)
new_df['CabinType']=new_df['Cabin'].apply(lambda x: x[0] if x is not None else None)

df_gb_pc_ct=new_df.groupby(['Pclass','CabinType'])[['Fare','Survived']].apply(lambda x:np.mean(x)).reset_index()

#将结果存入数据库
with sqlite3.connect('example_db') as conn:
    df_gb_pc_ct.to_sql('example_table_gb',con=conn,if_exists='replace',index=False)


## 小练习1


In [103]:
# sql 语句
"""
SELECT
    students.student_name,
    courses.course_name
from students join courses 
on students.student_id= courses.student_id
and (students.student_name='小小鲸' or courses.course_name='机器学习')
    
"""

"\nSELECT\n    students.student_name,\n    courses.course_name\nfrom students join courses \non students.student_id= courses.student_id\nand (students.student_name='小小鲸' or courses.course_name='机器学习')\n    \n"

## join 效率的提高

1. 索引
2. 


In [104]:
# 创建索引
query="""  
CREATE INDEX
if not exists
passengerIndex
on example_table(PassengerId)

"""
result=connection.execute(query).fetchall()
result

[]

In [105]:
# 快速创建一个索引
query = '''
CREATE INDEX                  -- CREATE INDEX： 关键字创建一个索引
IF NOT EXISTS                 -- IF NOT EXISTS: 如果该索引已存在，则不执行
passengerIndex                -- 索引名称,
ON example_table(PassengerId)        -- 索引对象,一般是某张表的某一列，形式为 table_name(column_name)
'''
result = connection.execute(query).fetchone()
result

In [106]:
# 创建完后直接利用索引
query=""" 
SELECT Name
from example_table
where PassengerId=2
"""

result=connection.execute(query).fetchall()
result

[('Cumings, Mrs. John Bradley (Florence Briggs Thayer)',)]

In [107]:

# 这里我们用 100 个相同查询的查询时间来看反映查询速度的变化

# 查询语句
# 统计每个 pclass 下乘客生存率的方差
query = '''
SELECT t1.Pclass
        ,avg((t2.Survived-t1.Survived)*(t2.Survived-t1.Survived)) AS std -- 方差计算
FROM example_table_gb AS t1
        JOIN example_table AS t2
            ON t1.Pclass = t2.Pclass 
GROUP BY t1.Pclass
'''

# before 记录创建索引前的执行时间
before = []
for i in range(100):
    start_time = time.time()
    _ = connection.execute(query).fetchall()
    before.append(time.time()-start_time)


OperationalError: no such column: t1.Survived